<a href="https://colab.research.google.com/github/JaiminiPatel067/AI-Email-Planner-Agent/blob/main/AI_Email_Planner_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install imapclient pyzmail36 transformer



ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer


Now Securely InPut your gmail Credetial

In [3]:
from getpass import getpass
import imaplib
import pyzmail

# --- Input credentials securely ---
username = input("Enter your Gmail address: ")
password = getpass("Enter your Gmail App Password: ")

imap_server = "imap.gmail.com"
imap_port = 993


Enter your Gmail address: jaiministudy@gmail.com
Enter your Gmail App Password: ··········


Connect to gmail IMAP server    

In [4]:
# Connect to Gmail
mail = imaplib.IMAP4_SSL(imap_server, imap_port)
mail.login(username, password)
mail.select("inbox")  # connect to inbox

print("IMAP login successful!")


IMAP login successful!


Fetch the mail

In [5]:
# Search for all emails
status, messages = mail.search(None, "ALL")
email_ids = messages[0].split()

print(f"Total emails in inbox: {len(email_ids)}")

# Fetch latest 5 emails
for email_id in email_ids[-5:]:
    status, msg_data = mail.fetch(email_id, "(RFC822)")
    raw_email = msg_data[0][1]

    msg = pyzmail.PyzMessage.factory(raw_email)

    subject = msg.get_subject()
    from_ = msg.get_addresses('from')

    # Get email body (text only)
    if msg.text_part:
        body = msg.text_part.get_payload().decode(msg.text_part.charset)
    else:
        body = ""

    print("\n---")
    print("From:", from_)
    print("Subject:", subject)
    print("Body Preview:", body[:200], "...\n")


Total emails in inbox: 700

---
From: [('Google', 'no-reply@accounts.google.com')]
Subject: Security alert for jenistudy2011@gmail.com
Body Preview: This is a copy of a security alert sent to jenistudy2011@gmail.com.
jaiministudy@gmail.com is the recovery email for this account. If you don't
recognize this account, remove
<https://accounts.goog ...


---
From: [('Google', 'no-reply@accounts.google.com')]
Subject: Security alert for jenistudy2011@gmail.com
Body Preview: This is a copy of a security alert sent to jenistudy2011@gmail.com.
jaiministudy@gmail.com is the recovery email for this account. If you don't
recognize this account, remove
<https://accounts.goog ...


---
From: [('Google', 'no-reply@accounts.google.com')]
Subject: Security alert for jenistudy2011@gmail.com
Body Preview: This is a copy of a security alert sent to jenistudy2011@gmail.com.
jaiministudy@gmail.com is the recovery email for this account. If you don't
recognize this account, remove
<https://accounts.goog ...

Set up SQLite Database

In [6]:
import sqlite3

# Connect to SQLite (file stored in Colab session)
conn = sqlite3.connect("emails_mvp.db")
cursor = conn.cursor()

# Create table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS emails (
    id INTEGER PRIMARY KEY,
    sender TEXT,
    subject TEXT,
    body TEXT,
    date TEXT
)
""")
conn.commit()


so again fetch email and store in SQLite Daatbase
one small issue in google colab is.. the gmail imap server is not connected for long term. if you get "abort: command: SEARCH => socket error: EOF error" error in this step then 1st you have to reconnect you gmail and then fetch the mails and store in sqlite db.

In [7]:
# Search for all emails
status, messages = mail.search(None, "ALL")
email_ids = messages[0].split()

# Fetch latest 10 emails for MVP
for email_id in email_ids[-10:]:
    status, msg_data = mail.fetch(email_id, "(RFC822)")
    raw_email = msg_data[0][1]
    msg = pyzmail.PyzMessage.factory(raw_email)

    sender = msg.get_addresses('from')[0][1] if msg.get_addresses('from') else ""
    subject = msg.get_subject()

    if msg.text_part:
        body = msg.text_part.get_payload().decode(msg.text_part.charset)
    else:
        body = ""

    date = msg.get_decoded_header('date')

    # Insert into SQLite
    cursor.execute("""
        INSERT INTO emails (sender, subject, body, date)
        VALUES (?, ?, ?, ?)
    """, (sender, subject, body, date))

conn.commit()
print("Fetched emails stored in SQLite successfully!")


Fetched emails stored in SQLite successfully!


Summarize emails using Hugging Face Transformers. so for that you have to install transformer library in your google colab.

In [8]:
from transformers import pipeline

# Use fast model
summarizer = pipeline("summarization", model="t5-small", device=0)  # use GPU if available

cursor.execute("SELECT id, body FROM emails")
emails = cursor.fetchall()

for email in emails:
    email_id, body = email
    if not body or len(body.strip()) == 0:
        continue

    # Skip summarization for short emails
    if len(body) < 150:
        summary = body[:150]  # take first 150 chars as "summary"
    else:
        # Cut text to make summarization fast
        text_to_summarize = body[:400]

        summary = summarizer(
            text_to_summarize,
            max_length=60,
            min_length=20,
            do_sample=False
        )[0]['summary_text']

    print(f"Email ID {email_id} Summary: {summary}\n")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

Email ID 17 Summary: this is a copy of a security alert sent to jenistudy2011@gmail.com . if you don't recognize this account, remove it .



Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Email ID 18 Summary: this is a copy of a security alert sent to jenistudy2011@gmail.com . if you don't recognize this account, remove it .



Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Email ID 19 Summary: KOSTENLOSER Standardversand bei Bestellungen ab 48€ https://stylevana.us2.list-manage.com/track/click?u=72ee6c80bc87844a386ceb320&id=035c053eb5&e=03dd6c8bd7)

Email ID 20 Summary: password created to sign in to your account jaiministudy@gmail.com . check activity 'https://accounts.google.com/AccountChooser?Email=jaiministudiey@yahoo.com'



Email Classification (Auto catergerize)

In [ ]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", return_all_scores=False))

categories = ["Work", "University", "Personal", "Finance", "Spam", "Action Required", "Reminder"]

def classify_email(body):
    result = classifier(body, categories)
    return result["labels"][0]  # highest probability


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

**Priority** Detection (High / Medium / Low)

In [ ]:
def detect_priority(email_body, subject):
    combined = subject + " " + email_body

    high_keywords = ["urgent", "deadline", "submit", "meeting", "important", "asap"]
    medium_keywords = ["please check", "follow up", "request"]

    text = combined.lower()

    if any(k in text for k in high_keywords):
        return "High"
    if any(k in text for k in medium_keywords):
        return "Medium"
    return "Low"


Summarize Emails (Fast version)

In [ ]:
summarizer = pipeline("summarization", model="t5-small")

def summarize_email(body):
    if len(body) < 150:
        return body[:150]  # no need summary
    return summarizer(body[:400], max_length=60, min_length=20)[0]['summary_text']


Store Everything in SQLite

In [ ]:
cursor.execute("ALTER TABLE emails ADD COLUMN category TEXT")
cursor.execute("ALTER TABLE emails ADD COLUMN priority TEXT")
cursor.execute("ALTER TABLE emails ADD COLUMN summary TEXT")
conn.commit()


Now process all emails:

In [ ]:
cursor.execute("SELECT id, subject, body FROM emails")
rows = cursor.fetchall()

for email in rows:
    row_id, subject, body = email

    category = classify_email(body)
    priority = detect_priority(body, subject)
    summary = summarize_email(body)

    cursor.execute("""
        UPDATE emails
        SET category=?, priority=?, summary=?
        WHERE id=?
    """, (category, priority, summary, row_id))

conn.commit()

print("Email planner classification + summary created successfully!")
